# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Classificação-de-Textos" data-toc-modified-id="Classificação-de-Textos-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Classificação de Textos</a></div><div class="lev2 toc-item"><a href="#Preâmbulo" data-toc-modified-id="Preâmbulo-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Preâmbulo</a></div><div class="lev2 toc-item"><a href="#Preparando-o-dataset" data-toc-modified-id="Preparando-o-dataset-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Preparando o dataset</a></div><div class="lev3 toc-item"><a href="#Buscando-o-texto-dos-livros-e-definindo-os-rótulos" data-toc-modified-id="Buscando-o-texto-dos-livros-e-definindo-os-rótulos-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Buscando o texto dos livros e definindo os rótulos</a></div><div class="lev3 toc-item"><a href="#Representando-as-palavras-através-de-índices-inteiros" data-toc-modified-id="Representando-as-palavras-através-de-índices-inteiros-122"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Representando as palavras através de índices inteiros</a></div><div class="lev3 toc-item"><a href="#Palavras-características-de-cada-livro" data-toc-modified-id="Palavras-características-de-cada-livro-123"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Palavras características de cada livro</a></div><div class="lev2 toc-item"><a href="#Dividindo-o-dataset-entre-treinamento-e-validação" data-toc-modified-id="Dividindo-o-dataset-entre-treinamento-e-validação-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Dividindo o dataset entre treinamento e validação</a></div><div class="lev3 toc-item"><a href="#Divisão-simples" data-toc-modified-id="Divisão-simples-131"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Divisão simples</a></div><div class="lev3 toc-item"><a href="#Divisão-para-uso-com-geradores-e-aumento-de-dados" data-toc-modified-id="Divisão-para-uso-com-geradores-e-aumento-de-dados-132"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Divisão para uso com geradores e aumento de dados</a></div><div class="lev4 toc-item"><a href="#Criando--geradores-para-treino-e-validação" data-toc-modified-id="Criando--geradores-para-treino-e-validação-1321"><span class="toc-item-num">1.3.2.1&nbsp;&nbsp;</span>Criando  geradores para treino e validação</a></div>

# Classificação de Textos

## Preâmbulo

In [14]:
%matplotlib inline
import matplotlib.pyplot as plot
from IPython import display
from __future__ import print_function

import os
import sys
import glob
import numpy as np
import numpy.random as nr

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.optimizers import (SGD, 
                              RMSprop, 
                              Adam, 
                              Adadelta, 
                              Adagrad)

sys.path.append('../src')
from my_keras_utilities import (get_available_gpus, 
                                load_model_and_history, 
                                save_model_and_history, 
                                TrainingPlotter)

os.makedirs('../../models',exist_ok=True)
np.set_printoptions(precision=3, linewidth=120)

In [15]:
import keras.backend as K

# K.set_image_data_format('channels_first')
K.set_floatx('float32')

print('Backend:        {}'.format(K.backend()))
print('Data format:    {}'.format(K.image_data_format()))
print('Available GPUS:', get_available_gpus())
print('Encoding:      ', sys.getdefaultencoding())

Backend:        tensorflow
Data format:    channels_last
Available GPUS: ['/gpu:0']
Encoding:       utf-8


## Preparando o dataset

### Buscando o texto dos livros e definindo os rótulos

In [16]:
data_dir = '../../datasets/'

autores = [
#     'Fernando_Sabino', 
    'Jorge_Amado',
    'Machado_de_Assis',
    'Erico_Verissimo',
]

book_text = []
book_author = []
book_title = []
for aut in autores:
    for fn in glob.glob(data_dir + 'livros/' + aut + '*.txt'):
        author, book = os.path.basename(fn).split('__')
        txt = open(fn, encoding='utf-8').read().replace('\x97', '')
        book_text.append(txt)
        book_author.append(author)
        book_title.append(book[:-4])
        print('{:7d}  {:18s} {}'.format(len(txt), author, book[:-4]))

author_list = list(set(book_author))
n_labels = len(author_list)
n_books = len(book_title)
book_label = [author_list.index(a) for a in book_author]
print('\n{} Labels:'.format(n_labels))
for i, autor in enumerate(author_list):
    print('    {:2d}: {}'.format(i, autor))

1001226  Jorge_Amado        Tereza_Batista_Cansada_de_Guerra
1030735  Jorge_Amado        Dona_flor_seus_dois_maridos
 427711  Jorge_Amado        Capitaes_de_Areia
 828417  Jorge_Amado        Gabriela
 352965  Machado_de_Assis   Memorias_Postumas_de_Bras_Cubas
 280683  Machado_de_Assis   Memorial_de_Aires
 411043  Machado_de_Assis   Esau_e_Jaco
 372459  Machado_de_Assis   Dom_Casmurro
 336677  Machado_de_Assis   Iaia_Garcia
 443778  Machado_de_Assis   Quincas_Borba
 337533  Machado_de_Assis   Helena
 294049  Erico_Verissimo    Clarissa
 890215  Erico_Verissimo    Incidente_em_Antares
 749265  Erico_Verissimo    O_Tempo_e_o_Vento_-_O_Continente
 699390  Erico_Verissimo    O_Tempo_e_o_Vento_-_O_Arquipelago

3 Labels:
     0: Machado_de_Assis
     1: Erico_Verissimo
     2: Jorge_Amado


### Representando as palavras através de índices inteiros

In [17]:
MAX_NB_WORDS = 20000

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(book_text)
sequences = tokenizer.texts_to_sequences(book_text)

w2i = tokenizer.word_index
i2w = dict([(v, k) for k, v in w2i.items()])

i2w_vec = np.array([i2w[i+1] for i in range(len(i2w))])

print('Found %d unique tokens.' % len(i2w))
print('Using the first %d words.' % max([max(s) for s in sequences]))

Found 61707 unique tokens.
Using the first 19999 words.


In [18]:
# escolhe e imprime uma sequencia aleatoria de cada livro
rseq_len = 100
for i, seq in enumerate(sequences):
    k = nr.randint(len(seq) - rseq_len)
    print('{}: {} -- {} palavras'.format(book_author[i], book_title[i], len(seq)).replace('_', ' '))
    print(' '.join([i2w[x] for x in seq[k:k+rseq_len]]), end='\n\n')


Jorge Amado: Tereza Batista Cansada de Guerra -- 164695 palavras
ouro no rio piauitinga sozinho ou em companhia de joão nascimento filho lá se ia para o rio de manhã cedinho — esse banho é saúde mestre joão ao regressar da primeira viagem após o aborto o doutor trouxe mil lembranças para tereza entre elas um maiô de banho — para irmos ao banho de rio — irmos os dois juntos — quis saber tereza pensativa — sim favo de mel os dois juntos tereza com o maiô por baixo do vestido o doutor com uma minúscula sob as calças atravessavam estância em direção ao rio apesar da hora matinal

Jorge Amado: Dona flor seus dois maridos -- 165877 palavras
espreguiçadeira pois bem dona flor descobri a malícia nas páginas mais ingênuas e força de sexo naquele barato e baixo dando outra dimensão aos o enredo a transformar dramalhão e personagens a virgem das campinas em marafona os m quase cresciam em brutais em vez de coleção menina e para adolescentes romances leitura para alcova o mesmo se passava com a ex

### Palavras características de cada livro

In [19]:
tfidf = tokenizer.sequences_to_matrix(sequences, mode='tfidf')
ww = np.argsort(tfidf, axis=1)[:, -8:]
print(i2w_vec[ww-1])

[['vavá' 'almério' 'januário' '—' 'dóris' 'brígida' 'justiniano' 'tereza']
 ['marilda' 'dinorá' 'teodoro' 'pelancchi' 'gisa' 'mirandão' 'rozilda' 'vadinho']
 ['trapiche' 'bedel' 'ester' 'barandão' 'almiro' 'dora' '–' 'pirulito']
 ['tuísca' 'tonico' 'ribeirinho' 'malvina' 'amâncio' 'fulgêncio' 'gabriela' 'nacib']
 ['loló' 'eusébia' 'cubas' 'borba' 'sabina' 'cotrim' 'marcela' 'virgília']
 ['carmo' 'libertos' 'prainha' 'noronha' 'cesária' 'aguiar' 'fidélia' 'tristão']
 ['coupé' 'gêmeos' 'excia' 'custódio' 'nóbrega' 'natividade' 'flora' 'cláudia']
 ['manduca' 'protonotário' 'bentinho' 'sancha' 'pádua' 'justina' 'escobar' 'capitu']
 ['procópio' 'valéria' 'jorge' 'garcia' 'madrasta' 'enteada' 'iaiá' 'estela']
 ['camacho' 'teófilo' 'tonica' 'borba' 'sofia' 'fernanda' 'benedita' 'rubião']
 ['ângela' 'eugênia' 'tomásia' 'melchior' 'helena' 'camargo' 'estácio' 'úrsula']
 ['gamaliel' 'dudu' 'eufrasina' 'tatá' 'belmira' 'tónico' 'zina' 'clarissa']
 ['campolargo' 'vacariano' 'vivaldino' 'getúlio' '

## Dividindo o dataset entre treinamento e validação

In [20]:
nr.seed(20170607)

batch_size  = 32
seq_size    = 200
valid_split = 0.2

### Divisão simples

In [21]:
all_data = [[] for i in range(n_labels)]

# divide cada livro em sequencias de 'seq_size' words
# 'all_data' contem as sequencias agrupadas por autor
for sequence, label in zip(sequences, book_label):
    n_seqs = len(sequence) // seq_size
    for i in range(n_seqs):
        beg = i * seq_size
        all_data[label].append(sequence[beg:beg+seq_size])

print('Sequencias obtidas para cada autor:', [len(x) for x in all_data])

# balanceando o dataset:
# calcula o numero de sequencias, N, de forma que o dataset 
# contenha N sequencias para cada autor
N = min([len(x) for x in all_data])
print('O dataset conterá {} sequencias por autor, totalizando {} sequencias.'.format(N, 3*N))

all_data = np.array([seq[:N] for seq in all_data], np.int32).reshape(-1, seq_size)
all_labels = np.array([[i] * N for i in range(n_labels)], np.int32).reshape(-1)
print('\nDataset shapes:', all_data.shape, all_labels.shape)


Sequencias obtidas para cada autor: [2143, 2189, 2701]
O dataset conterá 2143 sequencias por autor, totalizando 6429 sequencias.

Dataset shapes: (6429, 200) (6429,)


In [22]:
from sklearn.model_selection import train_test_split

Xtra, Xval, ytra, yval = train_test_split(all_data, all_labels, test_size=valid_split)
print(Xtra.shape, ytra.shape, Xval.shape, yval.shape)

(5143, 200) (5143,) (1286, 200) (1286,)


In [23]:
fn = data_dir + 'livros_sequences_{}.npz'.format(seq_size)
np.savez_compressed(fn, Xtra=Xtra, Xval=Xval, ytra=ytra, yval=yval, i2w=i2w_vec[:MAX_NB_WORDS])

### Divisão para uso com geradores e aumento de dados

In [24]:
valid_length = [int(0.2 * len(x)) for x in sequences]
valid_start = [nr.randint(2000, len(x) - 2000 - n) for x, n in zip(sequences, valid_length)]

valid_sequences = [seq[x0:x0+n] for seq, x0, n in zip(sequences, valid_start, valid_length)]

train_sequences = [seq[:x0] for seq, x0 in zip(sequences, valid_start)] + \
                  [seq[x0+n:] for seq, x0, n in zip(sequences, valid_start, valid_length)]

valid_labels = book_label
train_labels = book_label + book_label

n_train_words = sum([len(x) for x in train_sequences])
n_valid_words = sum([len(x) for x in valid_sequences])

print('Training sequences:')
print('-------------------')
for i, (seq, lab) in enumerate(zip(train_sequences, train_labels)):
    if i < n_books:
        print('{:2d}. {:16s} ({}) -- {:6d} palavras do início do livro {}'.format(i+1, book_author[i%n_books], lab,
                                                                                  len(seq), book_title[i%n_books]))
    else:
        print('{:2d}. {:16s} ({}) -- {:6d} palavras do final do livro  {}'.format(i+1, book_author[i%n_books], lab,
                                                                                  len(seq), book_title[i%n_books]))
print()
print('Validation sequences:')
print('---------------------')
for i, (seq, lab) in enumerate(zip(valid_sequences, valid_labels)):
    print('{:2d}. {:16s} ({}) -- {:6d} palavras do meio do livro {}'.format(i+1, book_author[i%n_books], lab,
                                                                            len(seq), book_title[i%n_books]))
print()
print('Total number of training words:  ', n_train_words)
print('Total number of validation words:', n_valid_words)

Training sequences:
-------------------
 1. Jorge_Amado      (2) --  11876 palavras do início do livro Tereza_Batista_Cansada_de_Guerra
 2. Jorge_Amado      (2) -- 120243 palavras do início do livro Dona_flor_seus_dois_maridos
 3. Jorge_Amado      (2) --  57529 palavras do início do livro Capitaes_de_Areia
 4. Jorge_Amado      (2) --  71632 palavras do início do livro Gabriela
 5. Machado_de_Assis (0) --  20278 palavras do início do livro Memorias_Postumas_de_Bras_Cubas
 6. Machado_de_Assis (0) --  13961 palavras do início do livro Memorial_de_Aires
 7. Machado_de_Assis (0) --  24553 palavras do início do livro Esau_e_Jaco
 8. Machado_de_Assis (0) --  27571 palavras do início do livro Dom_Casmurro
 9. Machado_de_Assis (0) --  15991 palavras do início do livro Iaia_Garcia
10. Machado_de_Assis (0) --  51304 palavras do início do livro Quincas_Borba
11. Machado_de_Assis (0) --  31535 palavras do início do livro Helena
12. Erico_Verissimo  (1) --   9346 palavras do início do livro Clarissa

#### Criando  geradores para treino e validação

In [25]:
class MyDataGenerator:
    def __init__(self, batch_size, seq_size, sequences, labels):
        self.batch_size = batch_size
        self.length = seq_size
        self.sequences = sequences
        self.labels = labels
        sizes = np.array([len(seq) for seq in sequences])
        self.p = 1.0 * sizes / sizes.sum()        # probabilidade de escolha para cada sequencia
        self.n = np.arange(len(sequences))        # indices de cada sequencia (para o choice abaixo)
        
    def __call__(self):
        while True:
            batch = np.empty((self.batch_size, self.length), np.int32)
            label = np.empty((self.batch_size, n_labels), np.int32)
            for i in range(self.batch_size):
                k = nr.choice(self.n, p=self.p)
                p = nr.randint(0, len(self.sequences[k]) - self.length)
                batch[i] = self.sequences[k][p:p+self.length]
                label[i] = to_categorical(self.labels[k], num_classes=n_labels)
            yield batch, label

            
train_gen = MyDataGenerator(batch_size, seq_size, train_sequences, train_labels)
valid_gen = MyDataGenerator(batch_size, seq_size, valid_sequences, valid_labels)


In [26]:
import pickle
fn = data_dir + 'livros_generators_{}.pkl'.format(seq_size)
pickle.dump([train_gen, valid_gen, i2w_vec[:MAX_NB_WORDS]], open(fn, 'wb'))